In [1]:
from pprint import pprint
import pandas as pd
import numpy as np
import json
import h5py
import glob
import matplotlib.pyplot as plt

from torch.utils.data import random_split

In [2]:
norm_dicc = {
    "l4s": {
        "B1": [0.0, 3.1057398816745447],
        "B2": [0.0, 19.76154895369598],
        "B3": [0.0, 31.551829580344585],
        "B4": [0.0, 33.16014987560649],
        "B5": [0.0, 9.972775876378055],
        "B6": [0.0, 4.144120061411549],
        "B7": [0.0, 3.6925308568047677],
        "B8": [0.0, 8.312890666119067],
        "B9": [0.0, 3.5485627183025263],
        "B10": [0.0, 21.44150144339325],
        "B11": [0.0, 5.786441765686274],
        "B12": [0.0, 19.661322873426624],
        "B13": [0.0, 4.040945090994509],
        "B14": [0.0, 5.121427202344607]
    },
    "own": {
        "B1": [681.0, 3165.0],
        "B2": [183.0, 8655.0],
        "B3": [258.0, 7697.0],
        "B4": [68.0, 6779.0],
        "B5": [1.0, 5293.0],
        "B6": [0.0, 5600.0],
        "B7": [58.0, 6544.0],
        "B8": [111.0, 8930.0],
        "B9": [36.0, 2972.0],
        "B10": [1.0, 270.0],
        "B11": [36.0, 7280.0],
        "B12": [1.0, 12937.0],
        "B13": [0.0, 1.4837092161178589],
        "B14": [0.000001, 4128.0]
    }
}
def read_data(path, test_size, seed):
    # TRAIN_PATH = f"{path}/img/*.h5" # data_val
    # TRAIN_MASK = f'{path}/mask/*.h5' # data_val
    # all_train = sorted(glob.glob(TRAIN_PATH))
    # all_mask = sorted(glob.glob(TRAIN_MASK))
    # train_size = int(0.8 * len(dataset))
    # val_size = len(dataset) - train_size
    # train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
    # x_train, x_valid, y_train, y_valid = train_test_split(all_train, all_mask, test_size=test_size, shuffle=True, random_state=seed)
    # return x_train, x_valid, y_train, y_valid

    from dataset import DatasetLandslide
    data_path = '/home/tidop/Desktop/projects/l4s/data/TrainData'
    dataset = DatasetLandslide(data_path)
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])


def normalize_minmax(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

class DataGenerator(Sequence):
    def __init__(self, all_train, all_mask, norm_dicc, batch_size=16):
        self.all_train = all_train
        self.all_mask = all_mask
        self.batch_size = batch_size
        self.n_imgs = len(all_train)
        self.norm = norm_dicc

    def __len__(self):
        return int(np.ceil(self.n_imgs / self.batch_size))

    def __getitem__(self, idx):
        batch_x = self.all_train[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.all_mask[idx * self.batch_size:(idx + 1) * self.batch_size]
        
        X = np.zeros((self.batch_size, 128, 128, 6))
        Y = np.zeros((self.batch_size, 128, 128, 1))
        
        for i, (img, mask) in enumerate(zip(batch_x, batch_y)):
            with h5py.File(img) as hdf:

                dict_norm = self.norm["l4s"] if "image" in img else self.norm["own"]
                
                data = np.array(hdf.get('img'))

                # assign 0 for the nan value
                data[np.isnan(data)] = 0.000001

                # ndvi calculation
                data_red = data[:, :, 3]
                data_nir = data[:, :, 7]
                data_ndvi = np.divide(data_nir - data_red, np.add(data_nir, data_red))
                
                # final array
                X[i, :, :, 0] = normalize_minmax(data[:, :, 1]) # , dict_norm["B2"]
                X[i, :, :, 1] = normalize_minmax(data[:, :, 2]) # , dict_norm["B3"]
                X[i, :, :, 2] = normalize_minmax(data[:, :, 3]) # , dict_norm["B4"]
                X[i, :, :, 3] = normalize_minmax(data[:, :, 12]) # , dict_norm["B13"]
                X[i, :, :, 4] = normalize_minmax(data[:, :, 13]) # , dict_norm["B14"]
                X[i, :, :, 5] = data_ndvi


            with h5py.File(mask) as hdf:
                data=np.array(hdf.get('mask'))
                Y[i, :, :, 0] = data

        X[np.isnan(X)] = 0.000001
        return X, Y

NameError: name 'Sequence' is not defined

In [3]:
with open("../models/model_resnet34_6b_ns.json", "r") as json_file:
    json_config = json_file.read()
model = tf.keras.models.model_from_json(json_config)
model.load_weights("../models/model_resnet34_6b_ns.h5")

2023-07-10 10:16:29.835997: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9539 MB memory:  -> device: 0, name: NVIDIA RTX A2000 12GB, pci bus id: 0000:65:00.0, compute capability: 8.6


In [4]:
SEED = 42
path_dataset = "../data/TrainData"
p_x_train, p_x_valid, p_y_train, p_y_valid = read_data(path_dataset, 0.3, SEED)
valid_generator = DataGenerator(p_x_valid, p_y_valid, norm_dicc, batch_size=16)

In [5]:
x_valid = valid_generator.__getitem__(0)[0]

In [6]:
x_valid.shape

(16, 128, 128, 6)

In [ ]:
threshold = 0.5
pred_img = model.predict(x_valid)
pred_img = (pred_img > threshold).astype(np.uint8)

In [ ]:
#@title Ver img { run: "auto" }
img = 12 #@param {type:"slider", min:0, max:113, step:1}
fig,(ax1,ax3)= plt.subplots(1,2,figsize=(15,10))
ax1.imshow(pred_img[img, :, :, 0])
ax1.set_title("Predictions")
# ax2.imshow(y_valid[img, :, :, 0])
# ax2.set_title("Label")
ax3.imshow(x_valid[img, :, :, 1:4])
ax3.set_title('Training Image')
# plt.savefig(f'graph/outputs/mod_564_6b/point_{img}.png')

## GEE

In [8]:
import ee
import geemap
ee.Initialize()

In [9]:
xmin,ymin,xmax,ymax = [-76.27886, -10.522948, -76.07424, -10.357505]
pol = ee.Geometry.Rectangle([xmin,ymin,xmax,ymax])

uriBase = 'gs://rgee_dev/COG/'
collection = ee.ImageCollection(ee.List([
    ee.Image.loadGeoTIFF(uriBase + 'AP_26505_FBS_F6970_RT1.cog.tif'),
    ee.Image.loadGeoTIFF(uriBase + 'AP_26505_FBS_F7000_RT1.cog.tif'),
    ee.Image.loadGeoTIFF(uriBase + 'AP_26505_FBS_F6960_RT1.cog.tif'),
    ee.Image.loadGeoTIFF(uriBase + 'AP_26505_FBS_F6990_RT1.cog.tif'),
    ee.Image.loadGeoTIFF(uriBase + 'AP_26505_FBS_F6980_RT1.cog.tif'),
    ee.Image.loadGeoTIFF(uriBase + 'AP_26082_FBS_F7000_RT1.cog.tif'),
    ee.Image.loadGeoTIFF(uriBase + 'AP_26082_FBS_F6990_RT1.cog.tif'),
    ee.Image.loadGeoTIFF(uriBase + 'AP_26505_FBS_F6950_RT1.cog.tif'),
    ee.Image.loadGeoTIFF(uriBase + 'AP_26082_FBS_F7010_RT1.cog.tif'),
    ee.Image.loadGeoTIFF(uriBase + 'AP_26082_FBS_F7020_RT1.cog.tif'),
    ee.Image.loadGeoTIFF(uriBase + 'AP_26082_FBS_F6980_RT1.cog.tif'),
    ee.Image.loadGeoTIFF(uriBase + 'AP_24988_FBD_F7010_RT1.cog.tif'),
    ee.Image.loadGeoTIFF(uriBase + 'AP_26082_FBS_F6970_RT1.cog.tif'),
    ee.Image.loadGeoTIFF(uriBase + 'AP_26082_FBS_F6970_RT1.cog.tif'),
    ee.Image.loadGeoTIFF(uriBase + 'AP_26257_FBS_F6970_RT1.cog.tif'),
    ee.Image.loadGeoTIFF(uriBase + 'AP_26257_FBS_F6960_RT1.cog.tif')
]))

dem = collection.filterBounds(pol).first().clip(pol).select("B0").clip(pol).toFloat().rename("B14")
slope = ee.Terrain.slope(dem).select("slope").rename("B13")

In [10]:
RGB_bands = ['B4','B3','B2'] #RGB
NDVI_bands = ['B8','B4']

d_s2 = ee.ImageCollection('COPERNICUS/S2_SR')\
              .filterDate('2020-06-01', '2020-07-30')\
              .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))\
              .filterBounds(pol)\
              .first()\
              .select("B1","B2","B3","B4","B5","B6","B7","B8","B8A","B9","B11","B12")\
              .clip(pol)
ndvi = d_s2.normalizedDifference(['B8','B4']).rename("B15")
dataset = d_s2.addBands(slope).addBands(dem).addBands(ndvi)

def normalize_minmax(data, band, norm_min, norm_max):
    band_img = data.expression(
        f'(b("{band}") - {norm_min})/({norm_max} - {norm_min})', {
            band: data.select(band)
        }).rename(band).toFloat().select(band)
    return band_img


In [11]:
band2 = normalize_minmax(dataset, "B2", norm_dicc["own"]["B2"][0], norm_dicc["own"]["B2"][1])
band3 = normalize_minmax(dataset, "B3", norm_dicc["own"]["B3"][0], norm_dicc["own"]["B3"][1])
band4 = normalize_minmax(dataset, "B4", norm_dicc["own"]["B4"][0], norm_dicc["own"]["B4"][1])
band13 = normalize_minmax(dataset, "B13", norm_dicc["own"]["B13"][0], norm_dicc["own"]["B13"][1])
band14 = normalize_minmax(dataset, "B14", norm_dicc["own"]["B14"][0], norm_dicc["own"]["B14"][1])

dataset_img = band2.addBands(band3).addBands(band4).addBands(band13).addBands(band14).addBands(ndvi)

In [24]:
# Map = geemap.Map()
# Map.centerObject(pol, 12)
# Map.addLayer(dataset_img.select("B14"), {'min': 0, 'max': 1}, 'dem')
# Map.addLayer(dataset_img.select("B15"), {'min': -0.2, 'max': 1}, 'NDVI')

In [ ]:
# Map = geemap.Map()
# Map.centerObject(pol, 12)
# Map.addLayer(dataset.select("B14").clip(pol), {'min': 0, 'max': 4000}, 'dem')
# Map.addLayer(dataset.select("B15").clip(pol), {'min': -0.2, 'max': 1}, 'NDVI')

In [12]:
outputBucket = 'rgee_dev'
imageFilePrefix = 'tesis/ld'
# Specify patch and file dimensions.
imageExportFormatOptions = {
  'patchDimensions': [128, 128],
  'compressed': True
}
# Setup the task.
imageTask = ee.batch.Export.image.toCloudStorage(
  image=dataset_img,
  description='Image Export',
  fileNamePrefix=imageFilePrefix,
  bucket=outputBucket,
  scale=10,
  fileFormat='TFRecord',
  region=pol.buffer(100).getInfo()['coordinates'],
  formatOptions=imageExportFormatOptions,
)
imageTask.start()

In [27]:
import time
while imageTask.active():
  print('Polling for task (id: {}).'.format(imageTask.id))
  time.sleep(5)

Polling for task (id: OFCU652MEA77UJAP4XP7W7P3).
Polling for task (id: OFCU652MEA77UJAP4XP7W7P3).
Polling for task (id: OFCU652MEA77UJAP4XP7W7P3).
Polling for task (id: OFCU652MEA77UJAP4XP7W7P3).
Polling for task (id: OFCU652MEA77UJAP4XP7W7P3).
Polling for task (id: OFCU652MEA77UJAP4XP7W7P3).
Polling for task (id: OFCU652MEA77UJAP4XP7W7P3).
Polling for task (id: OFCU652MEA77UJAP4XP7W7P3).
Polling for task (id: OFCU652MEA77UJAP4XP7W7P3).
Polling for task (id: OFCU652MEA77UJAP4XP7W7P3).
Polling for task (id: OFCU652MEA77UJAP4XP7W7P3).
Polling for task (id: OFCU652MEA77UJAP4XP7W7P3).
Polling for task (id: OFCU652MEA77UJAP4XP7W7P3).
Polling for task (id: OFCU652MEA77UJAP4XP7W7P3).
Polling for task (id: OFCU652MEA77UJAP4XP7W7P3).
Polling for task (id: OFCU652MEA77UJAP4XP7W7P3).
Polling for task (id: OFCU652MEA77UJAP4XP7W7P3).
Polling for task (id: OFCU652MEA77UJAP4XP7W7P3).
Polling for task (id: OFCU652MEA77UJAP4XP7W7P3).
Polling for task (id: OFCU652MEA77UJAP4XP7W7P3).
Polling for task (id

In [13]:
filesList = !gsutil ls 'gs://'{outputBucket}'/tesis/'
exportFilesList = [s for s in filesList if imageFilePrefix in s]
# Get the list of image files and the JSON mixer file.
imageFilesList = []
jsonFile = None
for f in exportFilesList:
  if f.endswith('.tfrecord.gz'):
    imageFilesList.append(f)
  elif f.endswith('.json'):
    jsonFile = f

In [14]:
# Load the contents of the mixer file to a JSON object.
jsonText = !gsutil cat {jsonFile}
# Get a single string w/ newlines from the IPython.utils.text.SList
mixer = json.loads(jsonText.nlstr)
pprint(mixer)

{'patchDimensions': [128, 128],
 'patchesPerRow': 17,
 'projection': {'affine': {'doubleMatrix': [10.0,
                                            0.0,
                                            359880.0,
                                            0.0,
                                            -10.0,
                                            8854970.0]},
                'crs': 'EPSG:32718'},
 'totalPatches': 238}


In [15]:
def predict_input_fn(fileNames,side,bands):

  # Read `TFRecordDatasets`
  dataset = tf.data.TFRecordDataset(fileNames, compression_type='GZIP')
  featuresDict = {x:tf.io.FixedLenFeature([side, side], dtype=tf.float32) for x in bands}

  # Make a parsing function
  def parse_image(example_proto):
    parsed_features = tf.io.parse_single_example(example_proto, featuresDict)
    return parsed_features

  def stack_images(features):
    nfeat = tf.transpose(tf.squeeze(tf.stack(list(features.values()))))
    return nfeat

  dataset = dataset.map(parse_image, num_parallel_calls=4)
  dataset = dataset.map(stack_images, num_parallel_calls=4)
  dataset = dataset.batch(side*side)
  return dataset

In [16]:
fileNames = imageFilesList
side = 128
bands = ['B2', 'B3', 'B4', 'B13', 'B14', 'B15']
dataset = tf.data.TFRecordDataset(fileNames[0], compression_type='GZIP')
featuresDict = {x:tf.io.FixedLenFeature([side, side], dtype=tf.float32) for x in bands}
predict_db = predict_input_fn(fileNames=imageFilesList, side=128, bands=["B2", "B3", "B4", "B13", "B14", "B15"])

In [91]:
# fileNames = imageFilesList
# side = 128

# # Read `TFRecordDatasets`
# bands = ['B2', 'B3', 'B4', 'B13', 'B14', 'B15']
# dataset = tf.data.TFRecordDataset(fileNames, compression_type='GZIP')
# featuresDict = {x:tf.io.FixedLenFeature([side, side], dtype=tf.float32) for x in bands}

# # Make a parsing function
# def parse_image(example_proto):
#     parsed_features = tf.io.parse_single_example(example_proto, featuresDict)
#     return parsed_features

# def stack_images(features):
#     nfeat = tf.transpose(tf.squeeze(tf.stack(list(features.values()))))
#     return nfeat

# dataset = dataset.map(parse_image, num_parallel_calls=4)
# dataset = dataset.map(stack_images, num_parallel_calls=4)
# dataset = dataset.batch(side*side)

In [38]:
predictions = model.predict(predict_db)

2023-07-06 20:58:22.434167: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 6442450944 exceeds 10% of free system memory.
2023-07-06 20:58:22.847721: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 249561088 exceeds 10% of free system memory.
2023-07-06 20:58:22.893798: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 265402368 exceeds 10% of free system memory.
2023-07-06 20:58:24.343346: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 249561088 exceeds 10% of free system memory.


1/1 [==============================] - 20s 20s/step


In [22]:
# Instantiate the writer.
PATCH_WIDTH , PATCH_HEIGHT = [128,128]
outputImageFile = 'gs://' + outputBucket + '/tesis/ld_out.TFRecord'
writer = tf.io.TFRecordWriter(outputImageFile)
curPatch = 1
for  prediction in predictions:
  patch = prediction.squeeze().T.flatten().tolist()

  if (len(patch) == PATCH_WIDTH * PATCH_HEIGHT):
    print('Done with patch ' + str(curPatch) + '...')
    # Create an example
    example = tf.train.Example(
      features=tf.train.Features(
        feature={
          'crop_prob': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=patch))
        }
      )
    )

    writer.write(example.SerializeToString())
    curPatch += 1
writer.close()

Done with patch 1...
Done with patch 2...
Done with patch 3...
Done with patch 4...
Done with patch 5...
Done with patch 6...
Done with patch 7...
Done with patch 8...
Done with patch 9...
Done with patch 10...
Done with patch 11...
Done with patch 12...
Done with patch 13...
Done with patch 14...
Done with patch 15...
Done with patch 16...
Done with patch 17...
Done with patch 18...
Done with patch 19...
Done with patch 20...
Done with patch 21...
Done with patch 22...
Done with patch 23...
Done with patch 24...
Done with patch 25...
Done with patch 26...
Done with patch 27...
Done with patch 28...
Done with patch 29...
Done with patch 30...
Done with patch 31...
Done with patch 32...
Done with patch 33...
Done with patch 34...
Done with patch 35...
Done with patch 36...
Done with patch 37...
Done with patch 38...
Done with patch 39...
Done with patch 40...
Done with patch 41...
Done with patch 42...
Done with patch 43...
Done with patch 44...
Done with patch 45...
Done with patch 46.

In [33]:
def predict_input_fn(fileNames, side, bands):

  # Read `TFRecordDatasets`
  dataset = tf.data.TFRecordDataset(fileNames, compression_type='GZIP')
  featuresDict = {x:tf.io.FixedLenFeature([side, side], dtype=tf.float32) for x in bands}

  # Make a parsing function
  def parse_image(example_proto):
    parsed_features = tf.io.parse_single_example(example_proto, featuresDict)
    return parsed_features

  def stack_images(features):
    nfeat = tf.transpose(tf.squeeze(tf.stack(list(features.values()))))
    return nfeat

  dataset = dataset.map(parse_image, num_parallel_calls=4)
  dataset = dataset.map(stack_images, num_parallel_calls=4)
  dataset = dataset.batch(side*side)
  return dataset

In [6]:
from google.cloud import storage

def list_blobs(bucket_name, prefix=None):
    """Lists all the blobs in the bucket."""
    storage_client = storage.Client()
    blobs = storage_client.list_blobs(bucket_name, prefix=prefix)
    return [blob.name for blob in blobs]

imageFilePrefix = 'tesis/ld'
filesList = list_blobs('rgee_dev', imageFilePrefix)

In [39]:
filesList

['tesis/ld.json', 'tesis/ld.tfrecord.gz', 'tesis/ld_out.TFRecord']

In [40]:
bucket_name = 'rgee_dev'
folder_name = 'tesis'
exportFilesList = [f"gs://{bucket_name}/{s}" for s in filesList if imageFilePrefix in s]
# Get the list of image files and the JSON mixer file.
imageFilesList = []
jsonFile = None
for f in exportFilesList:
    if f.endswith('.tfrecord.gz'):
        imageFilesList.append(f)
    elif f.endswith('.json'):
        jsonFile = f

In [41]:
imageFilesList

['gs://rgee_dev/tesis/ld.tfrecord.gz']

In [42]:
json_text = None
jsonFile = 'gs://rgee_dev/tesis/ld.json'
with tf.io.gfile.GFile(jsonFile, 'r') as f:
    json_text = f.read()
    mixer = json.loads(json_text)

In [55]:
mixer["projection"]["affine"]["doubleMatrix"]

[10.0, 0.0, 359880.0, 0.0, -10.0, 8854970.0]

In [43]:
with open("../models/model_resnet34_6b_ns.json", "r") as json_file:
    json_config = json_file.read()
model = tf.keras.models.model_from_json(json_config)
model.load_weights("../models/model_resnet34_6b_ns.h5")

In [44]:
fileNames = imageFilesList
side = 128
bands = ['B2', 'B3', 'B4', 'B13', 'B14', 'B15']
predict_db = predict_input_fn(fileNames=imageFilesList, side=128, bands=["B2", "B3", "B4", "B13", "B14", "B15"])

In [45]:
predictions = model.predict(predict_db)

1/1 [==============================] - 31s 31s/step


In [24]:
!gsutil ls -l {outputImageFile}

  15608754  2023-07-02T15:39:32Z  gs://rgee_dev/tesis/ld_out.TFRecord
TOTAL: 1 objects, 15608754 bytes (14.89 MiB)


In [2]:
# REPLACE WITH YOUR USERNAME:
USER_NAME = 'ryali93'
outputAssetID = 'users/' + USER_NAME + '/ld_out'
print('Writing to ' + outputAssetID)

Writing to users/ryali93/ld_out


In [26]:
# Start the upload. It step might take a while.
!earthengine upload image --asset_id={outputAssetID} {outputImageFile} {jsonFile}

Started upload task with ID: LI3VG7KQVKMCQ2H4IXJSR2WN


In [67]:
mixer

{'projection': {'crs': 'EPSG:32718',
  'affine': {'doubleMatrix': [10.0, 0.0, 359880.0, 0.0, -10.0, 8854970.0]}},
 'patchDimensions': [128, 128],
 'patchesPerRow': 17,
 'totalPatches': 238}

In [57]:
USER_NAME = 'ryali93'
NAME_OUT = 'ld_out_predictions'
outputAssetID = f'users/{USER_NAME}/{NAME_OUT}'
print('Writing to ' + outputAssetID)
# Instantiate the writer.
PATCH_WIDTH , PATCH_HEIGHT = [128,128]
outputImageFile = f'gs://{bucket_name}/{folder_name}/{NAME_OUT}_test.TFRecord'
writer = tf.io.TFRecordWriter(outputImageFile)
curPatch = 1
for  prediction in predictions:
    patch = prediction.squeeze().T.flatten().tolist()

    if (len(patch) == PATCH_WIDTH * PATCH_HEIGHT):
        print('Done with patch ' + str(curPatch) + '...')
        # Create an example
        example = tf.train.Example(
            features=tf.train.Features(
            feature={
                'ld_prob': 
                tf.train.Feature(
                    float_list=tf.train.FloatList(value=patch))
            })
        )

    writer.write(example.SerializeToString())
    curPatch += 1

writer.close()

Writing to users/ryali93/ld_out_predictions
Done with patch 1...
Done with patch 2...
Done with patch 3...
Done with patch 4...
Done with patch 5...
Done with patch 6...
Done with patch 7...
Done with patch 8...
Done with patch 9...
Done with patch 10...
Done with patch 11...
Done with patch 12...
Done with patch 13...
Done with patch 14...
Done with patch 15...
Done with patch 16...
Done with patch 17...
Done with patch 18...
Done with patch 19...
Done with patch 20...
Done with patch 21...
Done with patch 22...
Done with patch 23...
Done with patch 24...
Done with patch 25...
Done with patch 26...
Done with patch 27...
Done with patch 28...
Done with patch 29...
Done with patch 30...
Done with patch 31...
Done with patch 32...
Done with patch 33...
Done with patch 34...
Done with patch 35...
Done with patch 36...
Done with patch 37...
Done with patch 38...
Done with patch 39...
Done with patch 40...
Done with patch 41...
Done with patch 42...
Done with patch 43...
Done with patch 44.

In [66]:
import ee
ee.Initialize()
# outputAssetId = 'users/ryali93/ld_out_predictions'
outputAssetId = "projects/earthengine-legacy/assets/users/ryali93/ld_out_predictions",
gcs_image_uri = outputImageFile
# asset_request = {
# 'id': outputAssetId,
# 'tilesets': [
#     {
#     'sources': [{'primaryPath': gcs_image_uri}]
#     }
# ],
# 'crs': mixer["projection"]["crs"],
# 'affine': mixer["projection"]["affine"]["doubleMatrix"],
# }
params = {
    "name": outputAssetId,
    "tilesets": [{"sources": [{"uris": [outputImageFile]}]}]
}
ee.data.startIngestion(request_id=ee.data.newTaskId()[0], params=params)

EEException: Invalid JSON payload received. Unknown name "name" at 'image_manifest': Proto field is not repeating, cannot start list.

In [64]:
!earthengine upload image --asset_id={outputAssetID} {outputImageFile} {jsonFile}

usage: earthengine upload image [-h] [--wait [WAIT]] [--force]
                                [--asset_id ASSET_ID] [--last_band_alpha]
                                [--nodata_value NODATA_VALUE]
                                [--pyramiding_policy PYRAMIDING_POLICY]
                                [--bands BANDS] [--crs CRS]
                                [--manifest MANIFEST] [--property PROPERTY]
                                [--time_start TIME_START]
                                [--time_end TIME_END]
                                [src_files ...]

Uploads an image from Cloud Storage to Earth Engine. See docs for "asset set"
for additional details on how to specify asset metadata properties.

positional arguments:
  src_files             Cloud Storage URL(s) of the file(s) to upload. Must
                        have the prefix 'gs://'.

options:
  -h, --help            show this help message and exit
  --wait [WAIT], -w [WAIT]
                        Wait for the task to 